# Chapter-6: Cluster Analysis

## Unsupervised Learning
Machine Learning algorithms are often divided into two major types. Supervised learning and unsupervised learning algorithms. If the data is labeled, then we will apply a supervised learning algorithm to learn the patterns in the data. If the data is unlabelled, we need to find the patterns in an entirely different method using unsupervised learning algorithms.

There are two major algorithms of unsupervised learning
* The first algorithm PCA deals with the columns; it is known as Principal Component Analysis (PCA). In the PCA, we find a linear combination of the columns with maximum information.
* The second algorithm is cluster analysis. Cluster Analysis is used for customer segmentation. Cluster analysis is dividing the whole dataset into smaller subsets in such a way that customers inside a subset are very similar to each other.
* Cluster Analysis sounds very similar to decision trees. In the decision trees algorithm also, we are trying to divide the whole dataset into smaller subsets. Both cluster analysis and decision trees are used for customer segmentation; the only difference is data. The decision tree is a supervised learning algorithm, and cluster analysis is an unsupervised learning algorithm. Clustering is used in computer vision applications to divide the geospatial images into distinct regions and detect borders and objects.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
github_link="https://raw.githubusercontent.com/venkatareddykonasani/ML_DL_py_TF/master/Chapter6_Cluster/Datasets/Wholesale/"

### Cluster Analysis
* Dividing the data into clusters in such a way that within the cluster, all the elements are similar to each other, and they are dissimilar from elements in other clusters.
* Since we do not have the target variable in the data, we will use all the input variables for detecting the similarities to make the population into smaller subsets. These subsets are called clusters.
* We need to form the cluster in such a way that intra-cluster distance should be minimized, and inter-cluster distance should be maximized. Here distance means dissimilarity.

We will see it in detail with an example

The dataset name is “Wholesale customers Data Set”. It is publicly available data. It is available at https://archive.ics.uci.edu/ml/datasets/wholesale+customers.  The dataset has annual spending of customers on product categories like milk, groceries, frozen items, fresh items, detergents, and paper. Each row in the dataset represents one customer. Each column represents the type of product. The values in the data are annual aggregated spending in monitory units. 

In [ ]:
#cust_data = pd.read_csv(r"/content/drive/My Drive/DataSets/Chapter-6/datasets/Wholesale/Wholesale_customers_data.csv")
cust_data = pd.read_csv(github_link + "/Wholesale_customers_data.csv")

In [ ]:
print(cust_data.shape)
print(cust_data.columns.values)

In [ ]:
cust_data.info()

The dataset has 440 rows and nine columns. The column names are cust_id, channel, Region and rest of the columns are product types. Channel represents the type of customer. There are retail customers and commercial customers. Commercial customers are populated as HoReCa (Hotel-Restaurants-Café). The region is the customer region. We will see the individual variable descriptions to get more idea on the overall values. 

In [ ]:
pd.set_option('display.max_columns', None) 
cust_data.sample(n=5, random_state=77)

The above output shows a random sample of five records. We can use the random_state parameter to re-generate the same sample. Channel takes two values 1- HoReCa (Hotel-Restaurants-Café) and 2-retail. The region takes three values 1-Lisbon region, 2-Oporto region and 3-Other Region. The rest of the columns show the annual spending by different product types. 

In [ ]:
cust_data["Channel"].value_counts()
cust_data["Region"].value_counts()

In [ ]:
cust_data.describe()

From the above output, we need to ignore the Channel and Region because these two are categorical vriables. We will focus on the rest of the columns. The product category “fresh” is the most dominant. Spending on fresh products is highest when compared to other products. It is clear from the average and percentile values. Similarly, the least bought product is of type delicatessen. There are some outliers in each of these columns. We will look into them later. We will draw the box plot for all these five variables to get a visual representation of the above details. 


In [ ]:
plt.figure(figsize=(10,10))
plt.title("All the variables box plots", size=20)
sns.boxplot(x="variable", y="value", data=pd.melt(cust_data[['Fresh', 'Milk', 'Grocery','Frozen', 'Detergents_Paper', 'Delicatessen']]))
plt.show()

From the output, we can see that Fresh items are the most sold, followed by grocery and milk. Delicatessen is the lest sold items.  We got some basic idea on the data. Now we will see the overall objective 

#### Business problem and Objective
We want to send promotional offers to almost all customers. We do not want to send a single offer to every customer in the population. We want to divide the whole population into subsets and send a specific offer to the subsets independently. We want to apply cross-selling and upselling strategies. Cross-selling – If the customer buys product A, then sell him product B. Upselling – If a customer buys a product A then try to sell more of A.  Below are the overall objectives. 
1. Fresh items are the most sold items in the population. Is there a subset in the population where fresh items are sold less than other items like grocery or milk? We want to identify such segment, and send them offers on ‘Fresh items.” 
2. Find the group of customers with high spending on Fresh and low spending on Frozen, try cross-selling frozen for that group. 
3. Find a customer segment that spends very less on groceries and send them promotional offers on ‘Grocery.”

We are going to use a cluster analysis algorithm to solve the above problem. Let us go through the details of cluster Analysis. 


## Distance measure
The main idea of cluster analysis is to divide the whole population into subsets in such a way that the records inside a subset are similar to each other. We have first to understand what is similarity and dissimilarity. Let us take an example to understand the concept of dis-similarity. Take the sample data(five records) from the above customer spending data. 
* Scenario-1: Consider only a single column, “Fresh”. Looking at the output, find the two customers who are very similar to each other. 
* Scenario-2: Consider two columns “Fresh” and “Grocery. Looking these two column values find which of the two customers are similar to each other? 


In [ ]:
cust_data_sample=cust_data.sample(n=5, random_state=77)
cust_data_sample[["Cust_id","Fresh", "Grocery"]]

**Scenario-1:** Consider only a single column, “Fresh.” By looking at the output, we can see that customers with id 45 and 64 are very similar to each other. Customer 45 annual spending on fresh category is 5,181 and the closest spending in this simple is 4,760 and it is from customer id 64.

**Scenario-2:** Consider two columns “Fresh” and “Grocery”. In Scenario-1 it was easy for us to choose the similar customers. Now we need to consider two columns to choose the similar customers. The ids 45 and 64 are not closer to each other. Grocery has a really high value for customer 45. Let us consider these five customers as five points in the space. Let us draw these points by taking fresh on X-axis and Grocery on Y-axis. Below code helps us in drawing the graph


In [ ]:
plt.figure(figsize=(10,10))
plt.title("Fresh and Grocery spending plot", size=20)
plot=sns.scatterplot(x="Fresh",y="Grocery", data=cust_data_sample, s=500)
for i in list(cust_data_sample.index):
    plot.text(cust_data_sample.Fresh[i],cust_data_sample.Grocery[i],cust_data_sample.Cust_id[i],size=20)

We can see fresh on X-axis and grocery spending on Y-axis. Customer ids are shown as labels. From this graph, we can see that customer 64 and 224 are very close to each other. We can conclude that these two customers are similar to each other. We can confirm the same by looking at the data for 65 and 224. Since there are only five points here, we can draw them to see.  In real-time, datasets are large. We need a metric that can quantify the similarity and dissimilarity between two points.

We will use the Euclidean distance here

### Euclidean Distance
Let $(x_1, y_1)$ and $(x_2, y_2)$ be two points in the two-dimensional space. The distance between them is measured using the below formula

$D=\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$

The same can be extended to multiple dimensions  
$D=\sqrt{(x_2-x_1)^2+(y_2-y_1)^2+…+(k_2-k_1)^2}$

We can use this Euclidian distance formula to quantify the dissimilarity between any given customers. 


### Distance Matrix
* In our dataset, there are five customers. We need to find the distance of each customer from every other customer. The result will look like a matrix.
* The matrix that shows the distance of every customer in the population to every other customer is known as the distance matrix. 
* In our case, our distance matrix will be of size 5X5. If there are N customers, then the distance matrix will be of size NXN.
* The distance matrix size is independent of the number of columns in the dataset. All the columns are used in the calculation of distance. If a data matrix has a size of NXK, then the size of the distance matrix will be NXN
* The diagonal elements in the distance matrix are zeros. d(1,2) represents distance between customer-1 and customer-2 which is same as d(2,1). Distance matrix is a symmetric matrix and it has zeros in its principal diagonal. Let us create the distance matrix for our sample dataset. 


In [ ]:
def distance_cal(data_frame):
    distance_matrix=np.zeros((data_frame.shape[0],data_frame.shape[0]))
    for i in range(0 , data_frame.shape[0]):
        for j in range(0 , data_frame.shape[0]):
            distance_matrix[i,j]=round(np.sqrt(sum((data_frame.iloc[i] - data_frame.iloc[j])**2)))
    return(distance_matrix)

In the above code, we wrote a function that goes overall all the rows and calculates the distance from one row to every other row. Finally, the function returns the distance matrix as a two-dimensional array. 

In [ ]:
distance_matrix=distance_cal(cust_data_sample[["Fresh", "Grocery"]])
print(distance_matrix)
print(distance_matrix[0,0])
print(distance_matrix[1,0])
print(distance_matrix[2,1])

The customer ids are 46, 224, 65, 367, 289. In the matrix their indexes are 0,1,2,3,4. The distance matrix will return distances in the same order.  
* distance_matrix[1,0] is the distance between 224 and 46 which is equal to 16,441
* distance_matrix[2,1] is distance between 65 and 224 which is equal to 2,818

From the distance matrix, we can see that customer pair (65,224) are the closest; they are the most similar customers. Customers (289,46) are the farthest from each other they are the most dis-similar. 


The distance measure is the most important metric in cluster analysis. This metric will help us in quantifying the dis-similarity. Using this, we can now make the clusters. Euclidian distance is one of the most widely used distance measures.

## k-means clustering algorithm
The clustering algorithm takes the population as input and gives the customer segments as output. “K-Means” is the name of the clustering algorithm. It gives us K-clusters as output. We need to mention K while building the clusters. If we mention K=10, then ten clusters will be formed.  

**ALGORITHM**

* **Initialization Step**– Fix the number of clusters K. Randomly select K points from the data call them as cluster centers.  
* **Assignment step** - assign each data point to its closest cluster center
* **Update Step** - re-compute cluster centers by taking the average of all the records in a cluster. Update the cluster centers
* **Re-assignment step** - If cluster center changes then go back to assignment step
* **Stopping criteria** – Repeat steps 2,3 and 4 until there are no further changes in the cluster centers. 


#### Illustation
We will take a small sample of 30 records from the data and try to see the algorithm iterations. The code below tries to draw the state of the clusters at each iteration. While solving real-world business problems, we need not create these visualizations. 

In [ ]:
cust_data_sample2=cust_data.sample(n=30, random_state=99)
cust_data_sample2[["Cust_id","Fresh", "Grocery"]]

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Sample data", size=20)
plot=sns.scatterplot(x="Fresh",y="Grocery", data=cust_data_sample2, s=500)

In [ ]:
for i in range(1,4):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=4, max_iter=i, init='random', random_state=22)
    
    X=cust_data_sample2[["Fresh", "Grocery"]] # Custid is not needed
    kmeans = kmeans.fit(X) #Model building
    
    cust_data_sample2["Cluster_id"] = kmeans.predict(X)
    centers= kmeans.cluster_centers_
    print(centers)
    ##Scatter plot of customers
    plt.figure(figsize=(10,10))
    plt.title(i, size=20)
    sns.scatterplot(x="Fresh",y="Grocery", data=cust_data_sample2, s=500 , style="Cluster_id",hue="Cluster_id", palette=sns.hls_palette(4, l=.4, s=.8))
    plt.scatter(centers[:,0],centers[:,1], data=centers, s=500, marker="*")

In the above illustration the cluster ids are changing between the graphs. Colour and shapes change
based on the cluster ids. Ignore the colour code and shapes; focus on the four clusters in the above
graphs.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=333) 
X=cust_data.drop(['Cust_id', 'Channel', 'Region'],axis=1) 
kmeans = kmeans.fit(X) 

In the above code, we used kmeans.fit() function for building the model. We dropped irrelevant
columns from the data and supplied it to the fit() function. We have mentioned five clusters using
the n_clusters parameter. We use the random_state parameter to regenerate the same output
again. Otherwise, there will be minor changes in the output due to the random initialization.

In [ ]:
cust_data_clusters=cust_data
cust_data_clusters["Cluster_id"]= kmeans.predict(X)
cust_data_clusters.head(10)

Once the model is built, we get the predictions for each record by using predict() function. This function returns the cluster_id. We attach this cluster_id to the dataset.

With the output of K-means, we have added cluster_id to the data. Now we will analyze each
cluster. We will look at the cluster counts, and cluster means. The below code gives us cluster wise
count and mean.

In [ ]:
cluster_counts=cust_data_clusters['Cluster_id'].value_counts(sort=False)
cluster_means= cust_data_clusters.groupby(['Cluster_id']).mean()

In [ ]:
print(cluster_counts)
print(cluster_means)

From the output, we can see that five clusters are formed. Cluster ids are 0,1,2,3 and 4. Clusters id
zero has 217 records cluster-id four has 107 and cluster id two has 82 records. These three clusters
have more than fifty records. Cluster id three has just seven records. When we apply the clustering
algorithm on realtime datasets, this is how clusters are formed. We can NOT expect an equal
distribution of cluster counts. There will always be one or two clusters with several records and one
or two clusters with very few records.

In [ ]:
df_melt = pd.melt(cust_data_clusters.drop(['Cust_id', 'Channel', 'Region'],axis=1), "Cluster_id", var_name="Prod_type", value_name="Spend")

In the above code pd.melt() function transforms the data into three columns later it will be used in
drawing the graph.

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=df_melt)
plt.title("Cluster wise Spendings", size=20)

In [ ]:
Cluster=df_melt[(df_melt['Cluster_id']==0)] 
plt.figure(figsize=(7,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster)
plt.title("Cluster0", size=20)

In [ ]:
Cluster=df_melt[(df_melt['Cluster_id']==1)] 
plt.figure(figsize=(7,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster)
plt.title("Cluster1", size=20)

In [ ]:
Cluster=df_melt[(df_melt['Cluster_id']==2)] 
plt.figure(figsize=(7,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster)
plt.title("Cluster2", size=20)

In [ ]:
Cluster=df_melt[(df_melt['Cluster_id']==3)] 
plt.figure(figsize=(7,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster)
plt.title("Cluster3", size=20)

In [ ]:
Cluster=df_melt[(df_melt['Cluster_id']==4)] 
plt.figure(figsize=(7,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster)
plt.title("Cluster4", size=20)

In [ ]:
Cluster_2and3=df_melt[(df_melt['Cluster_id']==2)| (df_melt['Cluster_id']==3)]
plt.figure(figsize=(10,7))
sns.boxplot(x='Cluster_id', hue="Prod_type", y="Spend", data=Cluster_2and3)
plt.title("Cluster 2 and 3 only", size=20)

From this output, we can see the cluster wise spending in each cluster. We are now ready to solve
the wholesale data case study. Let us revisit the overall objectives stated in the problem statement.

### Deciding number of clusters
While building the model, we can choose the number of clusters based on our business knowledge.
Alternatively, we can follow a technique called the “elbow method” to get an idea of the optimal
number of clusters for a given dataset. Elbow method gives us a basic estimate on the number of
clusters, and we can then finetune and finalize the number of clusters.

#### Elbow method
While building the clusters, we have to make sure that intracluster distance should be minimum.
Intracluster distance is the distance of every point from its cluster center. We build several clusters
by taking multiple values of K. For each of these models. We calculate the cluster wise within-cluster
squared distance and sum it up. This value is known as Inertia. Intuitively, inertia is the Sum of
Squares of Errors (SSE)

In the elbow method, we draw a graph and decide the optimal value for K. The graph is drawn by
taking the number of clusters K on the X-axis and Intertia on the Y-axis. The graph looks like an
elbow.

In [ ]:
print(kmeans.inertia_)

We built five clusters we can check the value of inertia using the command “kmeans.inertia_”

In [ ]:
elbow_data=pd.DataFrame()
for i in range(1,16):
    kmeans_m2 = KMeans(n_clusters=i, random_state=333) # Mention the Number of clusters
    X=cust_data.drop(['Cust_id', 'Channel', 'Region'],axis=1) # Custid is not needed
    model= kmeans_m2.fit(X)
    elbow_data.at[i,"K"]=i
    elbow_data.at[i,"Inertia"]=round(model.inertia_)/10000000 #To lower the values
print(elbow_data)

The above code builds 15 models and stores the values of inertia ib elbow_data. Note that the value
of inertia are in billions, we have scaled them so that we can draw them in the graph.

We can see from the output that the inertia has reduced at a higher rate until the fourth or fifth
cluster. After that, the rate of decrement is too low. We will draw the elbow curve to verify and
finalize the optimal value of K.

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Elbow Plot", size=20)
plt.plot(elbow_data["K"],elbow_data["Inertia"],'--bD')
plt.xticks(elbow_data["K"])
plt.xlabel("K", size=15)
plt.ylabel("Inertia", size=15)

From the elbow plot, we can see the elbow at 5. We can choose 4 or 5 as the optimal number of
clusters for this data. This elbow estimation is just a suggested number for K. Depending on the
business problem we can choose a number we can choose an optimal number near this.